In [ ]:
!pip install --upgrade google-cloud-bigquery

To get the credential.json do the following:
1. Go to Google API -> choose BigQuery API
https://console.cloud.google.com/apis/dashboard
2. Create a project and activate the API
3. Go to login credentials and create a service Account
4. Download the credentials as json and save it in the data folder (we don't sync the data folder so its save)
5. Change the name of the credentials path to your id (or should we all have the same?)

In [91]:
from google.cloud import bigquery
from google.oauth2 import service_account

# Set up credentials and client
credentials_path = 'data/work-thesis-7b6f35ce5dfc.json'
credentials = service_account.Credentials.from_service_account_file(credentials_path)
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

# Example Query
query = """
    SELECT *
    FROM `bigquery-public-data.github_repos.sample_commits`
"""

# Execute the query
query_job = client.query(query)
df = query_job.to_dataframe()

# Displaying first few rows of the DataFrame
print(df.head())

                                     commit  \
0  d886e7ce157373773254f5c1e7f011cd9d3ec558   
1  6829e274a623187c24f7cfc0e3d35f25d087fcc5   
2  1d1ddf67dc3bfd80f60b216fa1fedfb242bee299   
3  146a6917fc30616401a090f55cff2b855ee5b2ab   
4  437184ae8bd1ef923a40b009e37801deae66ad55   

                                       tree  \
0  3a3ecc7af2269c41cff20eded1b9f7d97161ecfb   
1  a6ca873b29878ee7bc8c328af8a64bae0f49a37e   
2  e62ad9b0949e30bc058c8aea75ce11dcb38e72da   
3  b931b7a2d3e8eb049a129216767c66b8586bbdd9   
4  9b9fc2b295998135334c9a6fa32efff589486187   

                                       parent  \
0  [93a2f6329f40d951101ab8ab160f24680f1c122a]   
1  [6544e67bfb1bf55bcf3c0f6b37631917e9acfb74]   
2  [4b3dc9679cf779339d9049800803dfc3c83433d1]   
3  [0f2a86f200bc97ae6cefc5d3ac879094b3fcde48]   
4  [a96d6ef34751093797c3a6c6080733dd7af23d35]   

                                              author  \
0  {'name': 'Geert Uytterhoeven', 'email': 'a1ff8...   
1  {'name': 'Marek Szyprows

In [95]:
df.to_csv("data/sample_commits_full.csv")

In [92]:
df.committer[0]

{'name': 'Jonathan Corbet',
 'email': 'b2ce64d5587c02f12e367eac59751145a0660c51@lwn.net',
 'time_sec': 1459407515,
 'tz_offset': -360,
 'date': datetime.datetime(2016, 3, 31, 6, 58, 35, tzinfo=<UTC>)}

In [102]:
df.committer[0]["date"].month

3

In [104]:
df.committer[0]["date"].year

2016

In [105]:
import datetime
# Convert timestamp to datetime object
dt_object = df.committer[0]["date"]

# Extract the year
year = dt_object.year
month = dt_object.month

year, month

(2016, 3)

In [106]:
df.columns

Index(['commit', 'tree', 'parent', 'author', 'committer', 'subject', 'message',
       'trailer', 'difference', 'difference_truncated', 'repo_name',
       'encoding'],
      dtype='object')

In [ ]:
df.describe()

## Big Dataset: Proceed with caution
Going on the Big Dataset and trying to get all dara for April 2023
-> still no success

In [ ]:
#    WHERE 
#      EXTRACT(YEAR FROM TIMESTAMP_SECONDS(committer.date.seconds)) = 2023 AND 
#      EXTRACT(MONTH FROM TIMESTAMP_SECONDS(committer.date.seconds)) = 4;
query = """
    SELECT committer, repo_name, subject, message,
    FROM `bigquery-public-data.github_repos.commits`
    WHERE 
        EXTRACT(YEAR FROM TIMESTAMP_SECONDS(committer.date.seconds)) = 2023 AND 
        EXTRACT(MONTH FROM TIMESTAMP_SECONDS(committer.date.seconds)) = 4 AND
        EXTRACT(DAY FROM TIMESTAMP_SECONDS(committer.date.seconds)) BETWEEN 1 AND 2 AND
        RAND() <= 0.01
    LIMIT 5000;
"""

# Execute the query
query_job = client.query(query)
df_april = query_job.to_dataframe()

In [ ]:
df_april.committer[0]

In [ ]:
import datetime
# Convert timestamp to datetime object
dt_object = datetime.datetime.utcfromtimestamp(df_april.committer[0]["date"]["seconds"])

# Extract the year
year = dt_object.month
year

In [ ]:
import pandas as pd
# Extract unique repo names
unique_repo_names = list(pd.Series([repo for sublist in df_april['repo_name'] for repo in sublist]).unique())

# Extract unique committer names
unique_committer_names = list(df_april['committer'].apply(lambda x: x['name']).unique())

unique_repo_names, unique_committer_names